<a href="https://colab.research.google.com/github/CS20M038/CS6910-assignment_3/blob/main/InputEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import Libraries
from random import randint
from numpy import array
from numpy import argmax
import keras.backend as K
from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K

from scipy.ndimage.interpolation import shift
import csv
import random
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [ ]:
input_dict=['-PAD-']
target_dict=['-PAD-']
eng_alphabets = 'abcdefghijklmnopqrstuvwxyz'
pad_char = '-PAD-'

eng_alpha2index = {pad_char: 0}
for index, alpha in enumerate(eng_alphabets):
    eng_alpha2index[alpha] = index+1
    input_dict.append(alpha)

print(eng_alpha2index)
print(input_dict)
# Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)

hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_alpha2index = {pad_char: 0}
for index, alpha in enumerate(hindi_alphabets):
    hindi_alpha2index[alpha] = index+1
    target_dict.append(alpha)
    #print(alpha)

print(hindi_alpha2index)
print(target_dict)


# Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)

hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_index2alpha = {0: pad_char}
for index, alpha in enumerate(hindi_alphabets):
    hindi_index2alpha[index+1] = alpha
   # target_dict.append(alpha)
    #print(alpha)

print(hindi_index2alpha)

eng_alphabets = 'abcdefghijklmnopqrstuvwxyz'
eng_index2alpha = {0: pad_char}
for index, alpha in enumerate(eng_alphabets):
    eng_index2alpha[index+1] = alpha
   # target_dict.append(alpha)
    #print(alpha)

print(eng_index2alpha)

{'-PAD-': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
['-PAD-', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{'-PAD-': 0, 'ऀ': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'ऄ': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ऌ': 13, 'ऍ': 14, 'ऎ': 15, 'ए': 16, 'ऐ': 17, 'ऑ': 18, 'ऒ': 19, 'ओ': 20, 'औ': 21, 'क': 22, 'ख': 23, 'ग': 24, 'घ': 25, 'ङ': 26, 'च': 27, 'छ': 28, 'ज': 29, 'झ': 30, 'ञ': 31, 'ट': 32, 'ठ': 33, 'ड': 34, 'ढ': 35, 'ण': 36, 'त': 37, 'थ': 38, 'द': 39, 'ध': 40, 'न': 41, 'ऩ': 42, 'प': 43, 'फ': 44, 'ब': 45, 'भ': 46, 'म': 47, 'य': 48, 'र': 49, 'ऱ': 50, 'ल': 51, 'ळ': 52, 'ऴ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, 'ऺ': 59, 'ऻ': 60, '़': 61, 'ऽ': 62, 'ा': 63, 'ि': 64, 'ी': 65, 'ु': 66, 'ू': 67, 'ृ': 68, 'ॄ': 6

In [ ]:
def readXmlDataset(filename):
        tsv_file = open(filename)
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        lang1_words = []
        lang2_words = []

        for row in read_tsv:
            lang2_words.append(row[0])
            lang1_words.append(row[1])
            #print(row[0])

        return lang1_words, lang2_words
train_input_texts, train_target_texts = readXmlDataset('hi.translit.sampled.train.tsv')
test_input_texts, test_target_texts = readXmlDataset('hi.translit.sampled.test.tsv')

In [ ]:
def gt_integer_code(word, letter2index, device = 'cpu'):
    gt_rep = []
    #print(gt_rep)
    for letter_index, letter in enumerate(word):
        pos = letter2index[letter]
        gt_rep.append(pos)
    gt_rep.append(letter2index[pad_char])
    return gt_rep
#print(word_rep('abc',eng_alpha2index))
gt_integer_code('abc',eng_alpha2index)

[1, 2, 3, 0]

In [ ]:
def doc_integer_code(doc,letter2index):
    doc_integer_coded=[]
    for i, data in enumerate(doc):
        doc_integer_coded.append(gt_integer_code(data,letter2index))
    return doc_integer_coded
doc_integer_code(train_input_texts[0:5],eng_alpha2index)

[[1, 14, 0],
 [1, 14, 11, 7, 1, 14, 9, 20, 0],
 [21, 14, 3, 12, 5, 0],
 [1, 14, 11, 21, 18, 0],
 [1, 14, 11, 21, 18, 1, 14, 0]]

In [ ]:
encoded_docs=doc_integer_code(train_input_texts,eng_alpha2index)
max_length = 25
vocab_size = 27
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define the model
model_embed = Sequential()
model_embed.add(Embedding(vocab_size, 8, input_length=max_length))
model_embed.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model_embed.summary())
output_array = model_embed.predict(padded_docs)
print(output_array.shape)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 25, 8)             216       
Total params: 216
Trainable params: 216
Non-trainable params: 0
_________________________________________________________________
None
(44204, 25, 8)


In [ ]:
output_array[1]

array([[-0.03793967, -0.0124194 ,  0.0095832 ,  0.02267021,  0.04866118,
         0.00968114, -0.02523437, -0.02529996],
       [-0.04673539,  0.01019722,  0.02276536, -0.02871223, -0.03260108,
        -0.01375018, -0.04266194, -0.01933085],
       [-0.03664898,  0.04865488,  0.03286825, -0.04360212, -0.00460941,
         0.01637304, -0.03167987, -0.02501484],
       [ 0.01385424, -0.00999112,  0.01493018, -0.01882696, -0.00479795,
         0.03220458, -0.00337058, -0.0257418 ],
       [-0.03793967, -0.0124194 ,  0.0095832 ,  0.02267021,  0.04866118,
         0.00968114, -0.02523437, -0.02529996],
       [-0.04673539,  0.01019722,  0.02276536, -0.02871223, -0.03260108,
        -0.01375018, -0.04266194, -0.01933085],
       [ 0.01242529, -0.00254051,  0.00508539, -0.03158293, -0.0012924 ,
         0.04602196, -0.03291297,  0.00354161],
       [-0.02602918, -0.00411272,  0.02756573,  0.03020208,  0.04517576,
         0.03870547,  0.03335622,  0.03415782],
       [ 0.03326741,  0.00168619